# BERTのクラス分類で役職を推定するモデルを構成し、学習させる

## 1. モジュールのインポート

In [ ]:
import random
import glob
from tqdm import tqdm

import torch
from torch.utils.data import DataLoader
from transformers import BertJapaneseTokenizer, BertForSequenceClassification
import pytorch_lightning as pl

import pickle

In [ ]:
from aiwolf import Role,Agent

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"

## モデルの定義

In [ ]:
# 日本語の事前学習モデル
MODEL_NAME = 'cl-tohoku/bert-base-japanese-whole-word-masking'
MAX_LENGTH = 512
BATCH_SIZE = 128
# 文章をトークンに変換するトークナイザーの読み込み
tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)

In [ ]:
class BertForSequenceClassification_pl(pl.LightningModule):
    def __init__(self, model_name, num_labels,label_names, lr):
        # model_name: Transformersのモデルの名前
        # num_labels: ラベルの数
        # lr: 学習率

        super().__init__()
        
        # 引数のnum_labelsとlrを保存。
        # 例えば、self.hparams.lrでlrにアクセスできる。
        # チェックポイント作成時にも自動で保存される。
        self.save_hyperparameters()
        self.label_names = label_names

        # BERTのロード
        self.bert_sc = BertForSequenceClassification.from_pretrained(
            model_name,
            num_labels=num_labels
        )
        self.weight = torch.tensor([0.3, 1.0, 1.0, 1.0, 1.0,1.0,1.0,1.0]).cuda()
        
    # 学習データのミニバッチ(`batch`)が与えられた時に損失を出力する関数を書く。
    # batch_idxはミニバッチの番号であるが今回は使わない。
    def training_step(self, batch, batch_idx):
        output = self.bert_sc(**batch)
        #loss = output.loss
        logits = output['logits']
        #villagerの重みを下げる
        criterion = torch.nn.CrossEntropyLoss(weight=self.weight)
        loss = criterion(logits, batch['labels'])
                
        self.log('train_loss', loss) # 損失を'train_loss'の名前でログをとる。
        return loss
        
    # 検証データのミニバッチが与えられた時に、
    # 検証データを評価する指標を計算する関数を書く。
    def validation_step(self, batch, batch_idx):
        output = self.bert_sc(**batch)
        # val_loss = output.loss
        #loss = output.loss
        logits = output['logits']
        #villagerの重みを下げる
        criterion = torch.nn.CrossEntropyLoss(weight=self.weight)
        loss = criterion(logits, batch['labels'])
        self.log('val_loss', loss) # 損失を'val_loss'の名前でログをとる。

    # テストデータのミニバッチが与えられた時に、
    # テストデータを評価する指標を計算する関数を書く。
    def test_step(self, batch, batch_idx):
        labels = batch.pop('labels') # バッチからラベルを取得
        output = self.bert_sc(**batch)
        labels_predicted = output.logits.argmax(-1)
        num_correct = ( labels_predicted == labels ).sum().item()
        accuracy = num_correct/labels.size(0) #精度
        self.log('micro accuracy', accuracy) # 精度を'accuracy'の名前でログをとる。
        #各ラベルの精度を計算
        for i,label in enumerate(self.label_names):
            num_correct = ( labels_predicted[labels==i] == i ).sum().item()
            if labels[labels==i].size(0) == 0:
                each_accuracy = -1
            else:
                each_accuracy = num_correct/labels[labels==i].size(0)
            self.log(f"{label.name}_accuracy", each_accuracy) # 各ラベル名_accuracyの名前でログをとる。
            

    # 学習に用いるオプティマイザを返す関数を書く。
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.hparams.lr)

## 前処理

In [ ]:
dataset_for_loader = []

# データを取得
#現在のディレクトリを取得
import pathlib
from pathlib import Path

from aiwolfk2b.AttentionReasoningAgent.Modules.RoleEstimationModelPreprocessor import RoleEstimationModelPreprocessor
from aiwolfk2b.utils.helper import load_default_config
config = load_default_config()
preprocessor:RoleEstimationModelPreprocessor = RoleEstimationModelPreprocessor(config)
labels_list = preprocessor.role_label_list

current_dir = pathlib.Path().resolve()
#data_set_path=current_dir.joinpath("data","preprocessed_data").joinpath('dataset.pkl')
data_set_path=current_dir.joinpath("data","train",'dataset_small.pkl')

data_set_plain = pickle.load(open(data_set_path, 'rb'))
for data in data_set_plain:
    encoding = tokenizer(
            data[1],
            max_length=MAX_LENGTH, 
            padding='max_length',
            truncation=True
        )

    try:
        encoding['labels'] = labels_list.index(Role(data[0]))
    except:
        print(data[0],data[1])
        
    encoding = { k: torch.tensor(v) for k, v in encoding.items() }
    dataset_for_loader.append(encoding)

In [ ]:
# データセットの分割
random.shuffle(dataset_for_loader) # ランダムにシャッフル
n = len(dataset_for_loader)
n_train = int(0.6*n)
n_val = int(0.2*n)
dataset_train = dataset_for_loader[:n_train] # 学習データ
dataset_val = dataset_for_loader[n_train:n_train+n_val] # 検証データ
dataset_test = dataset_for_loader[n_train+n_val:] # テストデータ

# データセットからデータローダを作成
# 学習データはshuffle=Trueにする。
dataloader_train = DataLoader(
    dataset_train, batch_size=BATCH_SIZE//4, shuffle=True
) 
dataloader_val = DataLoader(dataset_val, batch_size=BATCH_SIZE)
dataloader_test = DataLoader(dataset_test, batch_size=BATCH_SIZE)

## 学習

In [ ]:
# 学習時にモデルの重みを保存する条件を指定
checkpoint = pl.callbacks.ModelCheckpoint(
    monitor='val_loss',
    mode='min',
    save_top_k=3,
    save_weights_only=True,
    dirpath='bert_role_estimation_model/',
)

# 学習の方法を指定
trainer = pl.Trainer(
    max_epochs=10,
    callbacks = [checkpoint]
)

In [ ]:
# PyTorch Lightningモデルのロード
model = BertForSequenceClassification_pl(
    MODEL_NAME, num_labels=len(labels_list),label_names=labels_list, lr=1e-5
)

# ファインチューニングを行う。
trainer.fit(model, dataloader_train, dataloader_val) 

In [ ]:
# 検証データで確認
best_model_path = checkpoint.best_model_path # ベストモデルのファイル
print('ベストモデルのファイル: ', checkpoint.best_model_path)
print('ベストモデルの検証データに対する損失: ', checkpoint.best_model_score)

In [ ]:
# テストデータで確認
test = trainer.test(dataloaders=dataloader_test)
print(f'Accuracy: {test[0]["micro accuracy"]:.2f}')
for i,label in enumerate(labels_list):
    print(f"{label.name}_accuracy: {test[0][f'{label.name}_accuracy']:.2f}")

In [ ]:
# Transformers対応のモデルを保存
out_model_dir = './bert_role_estimation_model'

import datetime
today = datetime.date.today()
out_filename = format(today, '%Y%m%d')

torch.save(model.bert_sc.state_dict(),out_model_dir + '/' + f'{out_filename}.pth')
model.bert_sc.save_pretrained('./bert_role_estimation_model') 

In [ ]:
# モデルのロード
best_model_path = "./bert_role_estimation_model/20230629.pth"
bert_sc_best = BertForSequenceClassification.from_pretrained(
            MODEL_NAME,
            num_labels=len(labels_list),
        )
bert_sc_best.load_state_dict(torch.load(best_model_path))

In [ ]:
#保存したモデルが取得できるか確認
bert_sc = BertForSequenceClassification.from_pretrained(
    './bert_role_estimation_model'
)
bert_sc = bert_sc.cuda()

In [ ]:
#学習したモデルの検証
test_input = ["""4,1,0,0,1,1,1,0
SEER
day1
talk:
もはもは＾－＾
もはもは＾－＾
もは＾－＾占いCO[02]たん◯れした＾－＾
もは＾－＾
もは＾－＾寒い所では…花は枯れてしまうの…；－；
もはよう＾－＾
占い把握＾－＾
[03]たん占い把握＾－＾
占いco[08]○
占い把握＾－＾
占い2把握＾－＾
[04]たんも占いね＾－＾
占い2把握ら＾－＾真狂か真狐とかかのあ＾－＾
人外全潜伏かしら＾－＾狂人はいなさそう？＾－＾
対抗把握しマス＾－＾
役職欠けて狂狐の可能性も＾－＾
>>13狂いない可能性もある＾－＾
対抗把握＾－＾狂>狼狐かな＾－＾
占い2出てるから全潜伏れはないれそ＾－＾真狂めかのあ＾－＾
[04]たんちょっと出方様子見っぽく思えたから真目下がるのあ；－；
狂人いなくて占い欠けの狼狐らったらやばえ；－；
>>17おんその場合は狼か狐が出てるのもあるなって＾－＾
とりま占い先宣言してほしいお＾－＾
今日はグレラン？＾－＾
グレーから柱出てもらう？＾－＾吊りあんもしゆゆうのいけお＾－＾
呪殺ないと真なのかまからん；－；
吊りは狐先に吊らないとら＾－＾
じゃあ[07]たん占う＾－＾
宣言したほうがいいかんじ？＾－＾
吊り余裕は銃殺出してもらえば増えるし対抗占いしてもらいたいかも＾－＾
じゃあ[06]たん行きます
漏れ吊っていいお＾－＾
day2
divine,1,4,HUMAN
talk:""","""4,1,0,0,1,1,1,0
VILLAGER
day1
talk:
もはもは＾－＾
もはもは＾－＾
もは＾－＾占いCO[01]たん◯れした＾－＾
もは＾－＾
もは＾－＾寒い所では…花は枯れてしまうの…；－；
もはよう＾－＾
占い把握＾－＾
[02]たん占い把握＾－＾
占いco[07]○
占い把握＾－＾
占い2把握＾－＾
[03]たんも占いね＾－＾
占い2把握ら＾－＾真狂か真狐とかかのあ＾－＾
人外全潜伏かしら＾－＾狂人はいなさそう？＾－＾
対抗把握しマス＾－＾
役職欠けて狂狐の可能性も＾－＾
>>13狂いない可能性もある＾－＾
対抗把握＾－＾狂>狼狐かな＾－＾
占い2出てるから全潜伏れはないれそ＾－＾真狂めかのあ＾－＾
[03]たんちょっと出方様子見っぽく思えたから真目下がるのあ；－；
狂人いなくて占い欠けの狼狐らったらやばえ；－；
>>17おんその場合は狼か狐が出てるのもあるなって＾－＾
とりま占い先宣言してほしいお＾－＾
今日はグレラン？＾－＾
グレーから柱出てもらう？＾－＾吊りあんもしゆゆうのいけお＾－＾
呪殺ないと真なのかまからん；－；
吊りは狐先に吊らないとら＾－＾
じゃあ[06]たん占う＾－＾
宣言したほうがいいかんじ？＾－＾
吊り余裕は銃殺出してもらえば増えるし対抗占いしてもらいたいかも＾－＾
じゃあ[05]たん行きます
漏れ吊っていいお＾－＾
"""]

truth_label = [Role.WEREWOLF, Role.VILLAGER]
test_input= [preprocessor.preprocess_text(raw) for raw in test_input]

encoding = tokenizer(
        test_input,
        max_length=MAX_LENGTH, 
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )
encoding = { k: v.cuda() for k, v in encoding.items() }

# 推論
with torch.no_grad():
    output = bert_sc.forward(**encoding)
scores = output.logits # 分類スコア
labels_predicted = scores.argmax(-1).cpu().numpy() # スコアが最も高いラベル
labels_predicted = [labels_list[i] for i in labels_predicted] # ラベルをラベル名に変換
for truth, predicted in zip(truth_label, labels_predicted):
    print(f"truth: {truth}, predicted: {predicted}")
    